In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, classification_report


# <h1 style='font-size:30px;'>Data Featuring</h1>

# <h2 style='font-size:25px;'>Import</h2>

In [3]:
X_train_full = pd.read_csv("/content/drive/My Drive/SoccerPrediction/data/X_train.csv", index_col=0)
X_test_full  = pd.read_csv("/content/drive/My Drive/SoccerPrediction/data/X_test.csv", index_col=0)


y_train = pd.read_csv("/content/drive/My Drive/SoccerPrediction/data/y_train.csv", index_col=0).squeeze()
y_test  = pd.read_csv("/content/drive/My Drive/SoccerPrediction/data/y_test.csv", index_col=0).squeeze()

# Make stripped-down versions for the model
non_features = ["Div", "Date",
                "HomeTeam_ShotOnTarget", "AwayTeam_ShotOnTarget"]

X_train_features = X_train_full.drop(columns=non_features)
X_test_features  = X_test_full.drop(columns=non_features)

# Save the feature order from the stripped-down version
feature_columns = X_train_features.columns
feature_columns.to_series(name="feature").to_csv("/content/drive/My Drive/SoccerPrediction/data/feature_columns.csv", index=False)

In [4]:
import pandas as pd

feat_order = pd.read_csv("/content/drive/My Drive/SoccerPrediction/data/feature_columns.csv")["feature"].tolist()

def align_features(df: pd.DataFrame, feat_order: list[str]) -> pd.DataFrame:
    # add any missing training columns as zeros
    missing = [c for c in feat_order if c not in df.columns]
    if missing:
        df = df.copy()
        for c in missing:
            df[c] = 0.0

    # drop any extras not used in training
    extra = [c for c in df.columns if c not in feat_order]
    if extra:
        df = df.drop(columns=extra, errors="ignore")

    # put in the exact training order
    df = df.reindex(columns=feat_order, fill_value=0.0)
    return df

# build the actual matrices the model will see
X_train = align_features(X_train_features, feat_order)
X_test  = align_features(X_test_features,  feat_order)

# optional safety check
assert list(X_train.columns) == feat_order == list(X_test.columns)



# <h2 style='font-size:25px;'>Probabilisitc Data Selection</h2>

In [5]:
def brier_score_multi(p, y):
    Y = pd.get_dummies(y).reindex(columns=[0,1,2], fill_value=0).values
    return float(np.mean(np.sum((p - Y)**2, axis=1)))

def group_brier_scores(X, y):
    rows = []
    # consensus: probabilities already
    if set(['pH_mean','pD_mean','pA_mean']).issubset(X.columns):
        P = X[['pA_mean','pD_mean','pH_mean']].values  # order: 0,1,2
        P = np.clip(P, 1e-6, 1-1e-6)
        P = P / P.sum(axis=1, keepdims=True)
        rows.append({'group':'consensus', 'brier': brier_score_multi(P, y)})
    # raw odds approximate probabilities via 1/odds then renormalize
    triplets = [('B365A','B365D','B365H'), ('PSA','PSD','PSH'), ('MaxA','MaxD','MaxH'), ("BWH","BWD","BWA"), ( "IWH","IWD","IWA"),
                ("WHH","WHD","WHA"), ("VCH","VCD","VCA"), ("PSCH","PSCD","PSCA")]
    for nameA,nameD,nameH in triplets:
        if {nameA,nameD,nameH}.issubset(X.columns):
            P = 1.0 / X[[nameA,nameD,nameH]].values
            P = np.clip(P, 1e-6, None)
            P = P / P.sum(axis=1, keepdims=True)
            rows.append({'group': f'odds_{nameH[:-1]}', 'brier': brier_score_multi(P, y)})
    return pd.DataFrame(rows).sort_values('brier')

brier_df = group_brier_scores(X_train, y_train)
print(brier_df)

       group     brier
0  consensus  0.557498
6    odds_WH  0.583697
8   odds_PSC  0.584901
7    odds_VC  0.585204
4    odds_BW  0.587706
5    odds_IW  0.589967
1  odds_B365  0.863573
2    odds_PS  0.865276
3   odds_Max  0.872169


Drop bad odds

In [6]:
exclude = ["B365H","B365D","B365A", "PSH","PSD","PSA", "MaxH","MaxD","MaxA"]
X_train = X_train.drop(columns=exclude)
X_test = X_test.drop(columns=exclude)

# <h2 style='font-size:25px;'>Data Selection</h2>

In [7]:
# ---- Define feature groups ----

# Form features
form_features = [
    "HomeTeam_points", "AwayTeam_points",
    "HomeTeam_avg_goal_diff", "AwayTeam_avg_goal_diff"
]

# Raw bookmaker odds (all H/D/A triplets)
bookmaker_odds = [
    "BWH","BWD","BWA",
    "IWH","IWD","IWA",
    "WHH","WHD","WHA",
    "VCH","VCD","VCA",
    "PSCH","PSCD","PSCA"
]

# Overrounds (one per bookmaker set)
overrounds = [
    "B365_overround","BW_overround","IW_overround","WH_overround",
    "VC_overround","Max_overround","PS_overround","PSC_overround"
]

# Elo ratings
elo_features = ["home_elo","away_elo","elo_diff"]

# Consensus features
consensus_features = ["pH_mean","pD_mean","pA_mean","overround_mean","overround_std"]

# Engineered extras
engineered_features = ["home_adv","draw_tightness"]

# <h3 style='font-size:20px;'>Informative groups</h3>

In [8]:
from sklearn.feature_selection import mutual_info_classif
import numpy as np
import pandas as pd

def group_mutual_info(X, y, groups):
    scores = []
    for gname, feats in groups.items():
        feats = [f for f in feats if f in X.columns]
        if not feats:
            continue
        mi = mutual_info_classif(X[feats], y, random_state=42, discrete_features=False)
        scores.append({
            'group': gname,
            'features': len(feats),
            'mi_mean': float(np.mean(mi)),
            'mi_median': float(np.median(mi)),
            'mi_top3_sum': float(np.sum(np.sort(mi)[-3:])),
        })
    return pd.DataFrame(scores).sort_values(['mi_top3_sum','mi_mean'], ascending=False)

groups = {
    'form': form_features,
    'bookmaker_odds': bookmaker_odds,
    'overrounds': overrounds,
    'elo': elo_features,
    'consensus': consensus_features,
    'engineered': engineered_features
}

mi_df = group_mutual_info(X_train, y_train, groups)
print(mi_df)

            group  features   mi_mean  mi_median  mi_top3_sum
1  bookmaker_odds        15  0.079875   0.100196     0.346286
4       consensus         3  0.080426   0.107707     0.241277
3             elo         3  0.059961   0.048379     0.179883
5      engineered         2  0.065186   0.065186     0.130372
0            form         4  0.023051   0.024665     0.084605
2      overrounds         8  0.011276   0.011363     0.063325


# <h3 style='font-size:20px;'>Group redundancy</h3>

In [9]:
def group_redundancy(X, groups):
    rows = []
    for gname, feats in groups.items():
        feats = [f for f in feats if f in X.columns]
        if len(feats) < 2:
            rows.append({'group': gname, 'intragroup_corr_mean': 0.0})
            continue
        corr = X[feats].corr().abs()
        upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))
        mean_corr = upper.stack().mean() if upper.size else 0.0
        rows.append({'group': gname, 'intragroup_corr_mean': float(mean_corr)})
    return pd.DataFrame(rows)

redundancy_df = group_redundancy(X_train, groups)
print(redundancy_df)

            group  intragroup_corr_mean
0            form              0.309717
1  bookmaker_odds              0.659672
2      overrounds              0.528862
3             elo              0.474944
4       consensus              0.531874
5      engineered              0.330012


# <h3 style='font-size:20px;'>Temporal Stability</h3>

In [10]:
import numpy as np
import pandas as pd

def psi_with_common_bins(a, b, bins=10):
    a = pd.Series(a, dtype='float64').dropna()
    b = pd.Series(b, dtype='float64').dropna()
    if len(a) < 20 or len(b) < 20:
        return 0.0
    s = pd.concat([a, b])
    if s.nunique(dropna=True) <= 2 or s.std() == 0:
        return 0.0

    qs = np.linspace(0, 1, bins + 1)
    edges = np.unique(np.nanquantile(s, qs))
    if len(edges) < 3:
        return 0.0

    ca = pd.cut(a, edges, include_lowest=True)
    cb = pd.cut(b, edges, include_lowest=True)

    pa = ca.value_counts(normalize=True, sort=False)
    pb = cb.value_counts(normalize=True, sort=False)

    idx = pa.index.union(pb.index)
    pa = pa.reindex(idx, fill_value=0).astype('float64') + 1e-6
    pb = pb.reindex(idx, fill_value=0).astype('float64') + 1e-6

    return float(np.sum((pa - pb) * np.log(pa / pb)))

def group_stability_no_date_common_bins(X, groups, split_ratio=0.7, bins=10, verbose=False):
    n = len(X)
    cut = int(n * split_ratio)
    early, late = X.iloc[:cut], X.iloc[cut:]

    # keep only numeric columns once to avoid repeated inference
    numeric_cols = set(X.select_dtypes(include=[np.number]).columns)

    rows = []
    for gname, feats in groups.items():
        feats = [f for f in feats if f in X.columns]
        feats = [f for f in feats if f in numeric_cols]  # ensure numeric
        psis, used = [], 0

        for f in feats:
            a = pd.to_numeric(early[f], errors='coerce')
            b = pd.to_numeric(late[f], errors='coerce')
            if a.dropna().empty or b.dropna().empty:
                continue
            try:
                psis.append(psi_with_common_bins(a.values, b.values, bins=bins))
                used += 1
            except Exception:
                continue

        psi_mean = float(np.mean(psis)) if used > 0 else 0.0  # default to 0 (stable) if none usable
        if verbose:
            print(f"{gname}: features={len(feats)}, used={used}, psi_mean={psi_mean:.4f}")

        rows.append({'group': gname, 'psi_mean': psi_mean, 'features': len(feats), 'used': used})

    return pd.DataFrame(rows).sort_values('psi_mean')

# Usage
groups = {
    'form': form_features,
    'bookmaker_odds': bookmaker_odds,
    'overrounds': overrounds,
    'elo': elo_features,
    'consensus': consensus_features,
    'engineered': engineered_features
}
stability_df = group_stability_no_date_common_bins(X_train, groups, split_ratio=0.7, bins=10, verbose=True)
print(stability_df)

form: features=4, used=4, psi_mean=0.0149
bookmaker_odds: features=15, used=15, psi_mean=0.0750
overrounds: features=8, used=8, psi_mean=4.3166
elo: features=3, used=3, psi_mean=0.1937
consensus: features=3, used=3, psi_mean=0.0850
engineered: features=2, used=2, psi_mean=0.1131
            group  psi_mean  features  used
0            form  0.014941         4     4
1  bookmaker_odds  0.075031        15    15
4       consensus  0.084992         3     3
5      engineered  0.113055         2     2
3             elo  0.193702         3     3
2      overrounds  4.316615         8     8


# <h3 style='font-size:20px;'>Cross Group Correlation</h3>

In [11]:
def cross_group_corr(X, groups):
    # mean absolute correlation between every pair of groups
    names = list(groups.keys())
    data = []
    for i in range(len(names)):
        for j in range(i+1, len(names)):
            gi = [f for f in groups[names[i]] if f in X.columns]
            gj = [f for f in groups[names[j]] if f in X.columns]
            if not gi or not gj:
                continue
            corr = X[gi+gj].corr().abs().loc[gi, gj].values
            data.append({'g1':names[i], 'g2':names[j], 'mean_abs_corr': float(np.nanmean(corr))})
    return pd.DataFrame(data).sort_values('mean_abs_corr')

overlap_df = cross_group_corr(X_train, groups)
print(overlap_df.head(10))

                g1              g2  mean_abs_corr
1             form      overrounds       0.023592
5   bookmaker_odds      overrounds       0.039066
10      overrounds       consensus       0.039583
11      overrounds      engineered       0.046445
9       overrounds             elo       0.104737
4             form      engineered       0.376694
2             form             elo       0.387839
0             form  bookmaker_odds       0.400544
3             form       consensus       0.402658
13             elo      engineered       0.575139


# <h3 style='font-size:20px;'>Rank Group</h3>

In [12]:
import pandas as pd
import numpy as np

def rank_groups_multi(mi_df, redundancy_df, overlap_df, stability_df,
                      mi_col="mi_top3_sum",
                      weights=None,          # e.g. {"mi":2, "redundancy":1, "overlap":1, "stability":1}
                      sort_by="overall"):    # "overall", "mi", "redundancy", "overlap", or "stability"
    """
    Combine group diagnostics and produce ranks per criterion (and optional overall rank).

    Parameters
    ----------
    mi_df : DataFrame with columns ["group", mi_col] where mi_col is e.g. "mi_top3_sum"
    redundancy_df : DataFrame with columns ["group", "intragroup_corr_mean"]
    overlap_df : DataFrame with columns ["g1","g2","mean_abs_corr"] (pairwise group overlaps)
    stability_df : DataFrame with columns ["group","psi_mean"]
    mi_col : str, column name in mi_df to use for MI ranking
    weights : dict or None, weights for overall rank (keys: "mi","redundancy","overlap","stability")
    sort_by : str, which rank to sort by ("overall" requires weights)

    Returns
    -------
    DataFrame with raw metrics, ranks per criterion, and optional overall rank.
    """

    # --- Cross-group overlap → per-group mean overlap (symmetrize pairs) ---
    if overlap_df is not None and not overlap_df.empty:
        og = pd.concat([
            overlap_df.rename(columns={"g1": "group", "g2": "other"}),
            overlap_df.rename(columns={"g2": "group", "g1": "other"})
        ], ignore_index=True)
        cross_mean = (og.groupby("group")["mean_abs_corr"]
                        .mean()
                        .rename("crossgroup_corr_mean")
                        .reset_index())
    else:
        cross_mean = pd.DataFrame(columns=["group", "crossgroup_corr_mean"])

    # --- Merge everything on 'group' ---
    cols_mi = ["group", mi_col]
    cols_rd = ["group", "intragroup_corr_mean"]
    cols_st = ["group", "psi_mean"]

    df = (mi_df[cols_mi]
            .merge(redundancy_df[cols_rd], on="group", how="outer")
            .merge(cross_mean, on="group", how="outer")
            .merge(stability_df[cols_st], on="group", how="outer"))

    # Helper to rank while pushing NaNs to the bottom (worst)
    def _rank(series, ascending):
        s = series.copy()
        if s.isna().any():
            fill = (s.max() + 1) if ascending else (s.min() - 1)
            s = s.fillna(fill)
        return s.rank(ascending=ascending, method="min")

    # --- Per-criterion ranks ---
    df["rank_mi"]          = _rank(df[mi_col], ascending=False)              # higher MI is better
    df["rank_redundancy"]  = _rank(df["intragroup_corr_mean"], ascending=True)
    df["rank_overlap"]     = _rank(df["crossgroup_corr_mean"], ascending=True)
    df["rank_stability"]   = _rank(df["psi_mean"], ascending=True)

    # --- Optional overall rank (weighted sum of ranks; lower = better) ---
    if weights:
        w_mi  = float(weights.get("mi", 1.0))
        w_rd  = float(weights.get("redundancy", 1.0))
        w_ov  = float(weights.get("overlap", 1.0))
        w_ps  = float(weights.get("stability", 1.0))
        df["overall_rank_score"] = (
            w_mi*df["rank_mi"] +
            w_rd*df["rank_redundancy"] +
            w_ov*df["rank_overlap"] +
            w_ps*df["rank_stability"]
        )
        df["rank_overall"] = df["overall_rank_score"].rank(ascending=True, method="min")

    # --- Sorting ---
    sort_map = {
        "overall": "rank_overall",
        "mi": "rank_mi",
        "redundancy": "rank_redundancy",
        "overlap": "rank_overlap",
        "stability": "rank_stability",
    }
    sort_col = sort_map.get(sort_by, "rank_mi")
    if sort_by == "overall" and not weights:
        sort_col = "rank_mi"  # fallback if weights not provided

    # Nice column order
    out_cols = ["group", mi_col, "intragroup_corr_mean", "crossgroup_corr_mean", "psi_mean",
                "rank_mi", "rank_redundancy", "rank_overlap", "rank_stability"]
    if weights:
        out_cols += ["overall_rank_score", "rank_overall"]

    return df[out_cols].sort_values(sort_col, ascending=True).reset_index(drop=True)

In [13]:
# Choose weights (tweak as you like)
weights = {"mi": 2.0, "redundancy": 1.0, "overlap": 1.0, "stability": 1.0}

ranked = rank_groups_multi(
    mi_df=mi_df,
    redundancy_df=redundancy_df,
    overlap_df=overlap_df,
    stability_df=stability_df,   # from your PSI-with-common-bins function
    mi_col="mi_top3_sum",
    weights=weights,
    sort_by="overall"            # or "mi"/"redundancy"/"overlap"/"stability"
)
print(ranked)

            group  mi_top3_sum  intragroup_corr_mean  crossgroup_corr_mean  \
0            form     0.084605              0.309717              0.318265   
1  bookmaker_odds     0.346286              0.659672              0.480782   
2      engineered     0.130372              0.330012              0.462187   
3       consensus     0.241277              0.531874              0.482342   
4             elo     0.179883              0.474944              0.463766   
5      overrounds     0.063325              0.528862              0.050685   

   psi_mean  rank_mi  rank_redundancy  rank_overlap  rank_stability  \
0  0.014941      5.0              1.0           2.0             1.0   
1  0.075031      1.0              6.0           5.0             2.0   
2  0.113055      4.0              2.0           3.0             4.0   
3  0.084992      2.0              5.0           6.0             3.0   
4  0.193702      3.0              3.0           4.0             5.0   
5  4.316615      6.0       

In [14]:
from sklearn.preprocessing import LabelEncoder

feat_cats  = ["HomeTeam", "AwayTeam"]
feat_nums  = form_features
feat_all   = feat_cats + feat_nums + bookmaker_odds

Xtr = X_train[feat_all].copy()
Xte = X_test[feat_all].copy()
ytr, yte = y_train, y_test

for c in feat_cats:
    Xtr[c] = Xtr[c].astype(str)
    Xte[c] = Xte[c].astype(str)


In [15]:
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTENC
from collections import Counter

# Define feature groups
feat_cats = ["HomeTeam", "AwayTeam"]
feat_nums = form_features
feat_all = feat_cats + feat_nums + bookmaker_odds

# Prepare data
Xtr = X_train[feat_all].copy()
Xte = X_test[feat_all].copy()
ytr, yte = y_train, y_test

# Ensure categorical features are strings
for c in feat_cats:
    Xtr[c] = Xtr[c].astype(str)
    Xte[c] = Xte[c].astype(str)

# --- ALWAYS encode categorical features first ---
encoders = {}
Xtr_enc = Xtr.copy()
Xte_enc = Xte.copy()

for c in feat_cats:
    le = LabelEncoder()
    # Fit on training data only, transform both train and test
    Xtr_enc[c] = le.fit_transform(Xtr[c])
    Xte_enc[c] = le.transform(Xte[c])  # Use transform, not fit_transform
    encoders[c] = le

In [16]:
# Get indices of categorical columns for SMOTENC
cat_idx = [Xtr_enc.columns.get_loc(c) for c in feat_cats]

# --- Apply SMOTENC ---
smote = SMOTENC(
    categorical_features=cat_idx,
    sampling_strategy="not majority",  # upsample draws & away, keep home
    random_state=42
)
Xtr_bal, ytr_bal = smote.fit_resample(Xtr_enc, ytr)

print("Before SMOTE:", Counter(ytr))
print("After SMOTE:", Counter(ytr_bal))

# --- For CatBoost: Keep encoded features ---
# CatBoost works with encoded categorical features
Xtr_final = pd.DataFrame(Xtr_bal, columns=Xtr_enc.columns)
Xte_final = Xte_enc.copy()

Before SMOTE: Counter({2: 2013, 0: 1178, 1: 989})
After SMOTE: Counter({2: 2013, 0: 2013, 1: 2013})


In [17]:
from sklearn.utils import shuffle
Xtr_bal, ytr_bal = shuffle(Xtr_bal, ytr_bal, random_state=42)

In [18]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, precision_score

def run_model(classifier, param_grid, X_train, y_train, X_test, y_test):

    grid_search = GridSearchCV(estimator=classifier, param_grid=param_grid, scoring='balanced_accuracy',
                               cv=5, verbose=1)
    grid_search.fit(X_train, y_train)
    model = grid_search.best_estimator_
    y_pred = model.predict(X_test)
    print(f"Test Set Accuracy: {accuracy_score(y_test, y_pred):.2f}")
    print(classification_report(y_test, y_pred, zero_division=0))

    return grid_search.best_params_

In [19]:
!pip install skorch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.1/263.1 kB 7.9 MB/s eta 0:00:00


In [20]:
import random

random.seed(42)
np.random.seed(42)

import torch
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [21]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, WeightedRandomSampler

# Convert to tensors
Xtr = torch.tensor(np.asarray(Xtr_enc), dtype=torch.float32)
ytr_t = torch.tensor(np.asarray(ytr), dtype=torch.long)
Xte = torch.tensor(np.asarray(Xte_enc), dtype=torch.float32)
yte_t = torch.tensor(np.asarray(yte), dtype=torch.long)

train_ds = TensorDataset(Xtr, ytr_t)
test_ds = TensorDataset(Xte, yte_t)

train_loader = DataLoader(TensorDataset(Xtr, ytr_t), batch_size=512, shuffle=True)
test_loader  = DataLoader(TensorDataset(Xte, yte_t), batch_size=256, shuffle=False)

In [22]:
input_dim = Xtr.shape[1]
num_classes = int(ytr_t.max().item() + 1)

class MLP(nn.Module):
    def __init__(self, d_in, d_out):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(d_in, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, d_out)
        )
    def forward(self, x):
        return self.net(x)

In [23]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [30]:
# Train

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MLP(input_dim, num_classes).to(device)

epochs = 50
class_weights = torch.tensor([1.0, 1.5, 1.0], dtype=torch.float32, device=device)
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-3, weight_decay=1e-5)

torch.manual_seed(42)
model.train()
for epoch in range(epochs):
    epoch_loss = 0.0
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        logits = model(xb)
        loss = criterion(logits, yb)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item() * xb.size(0)

In [31]:
model.eval()
correct = 0
total = 0
all_probs = []
all_preds = []
with torch.no_grad():
    for xb, yb in test_loader:
        xb, yb = xb.to(device), yb.to(device)
        logits = model(xb)
        probs = torch.softmax(logits, dim=1)
        preds = probs.argmax(dim=1)
        correct += (preds == yb).sum().item()
        total += yb.size(0)
        all_probs.append(probs.cpu())
        all_preds.append(preds.cpu())

proba = torch.cat(all_probs, dim=0).numpy()
y_pred = torch.cat(all_preds, dim=0).numpy()

print(f"Test Set Accuracy: {accuracy_score(yte, y_pred):.2f}")
print(classification_report(yte, y_pred, zero_division=0))

Test Set Accuracy: 0.54
              precision    recall  f1-score   support

           0       0.53      0.40      0.46        47
           1       0.40      0.24      0.30        50
           2       0.58      0.80      0.67        83

    accuracy                           0.54       180
   macro avg       0.50      0.48      0.48       180
weighted avg       0.52      0.54      0.51       180



In [32]:
torch.jit.script(model).save("mlp_model.ts")

In [ ]:
import torch
model = torch.jit.load("mlp_model.ts", map_location="cpu")
print("Loaded successfully")

Loaded successfully
